
## Imports

In [235]:
import pandas as pd
import random

In [236]:
import sys
from docplex.cp.model import CpoModel
mdl = CpoModel()
w = mdl.integer_var_list(5,min=0,max=25,name="w")

In [237]:
# convert a word represented as an array of chars into a string
def convert_array_to_string(w):
    str = ""
    for i in range(len(w)):
        str += w[i]
    return str

# get a set of words as strings 
def get_set_of_words(fileName):
    words_raw = pd.read_csv(fileName)
    words = words_raw['words']
    Words = set()
    for i in range(len(words)):
        if len(words[i]) > 5:
            continue
        Words.add(convert_array_to_string(words[i]))
    return Words

In [238]:
# this function returns a guess that is consistent with the prior history of guesses and the set of Words
# input:
#       a set of words represented as strings
#       a history of prior guesses and their assessments: 
#       the history is an array whose elements are (guess,nb_correct,nb_present) where
#       guess is a string, nb_correct is the number of correct letters and nb_position is the number of letters 
#       that are present but incorrectly placed. 
# output:
#       a new guess as a string

def guess(Words,history):
    numberArray = []
    wordArray = []
    for a in Words:
        for c in a:
            Order = ord(c)-97
            numberArray.append(Order)
        wordArray.append(numberArray)
        numberArray = []
    mdl.add(mdl.allowed_assignments(w,wordArray)) # Allow combination of the dvar word with number array
    
    pastGuess = []
    numberlist = []
    historylist = []
    for h in range(len(history)):
        pastGuess.append(history[h][0])
    for g in pastGuess:
        for c in g:
            historylist.append(ord(c)-97)
        historylist.append(numberlist)
        numberlist = []
    mdl.add(mdl.forbidden_assignments(w,historylist)) # Prohibit the repetition of guess. 
    
    for h in range(len(history)):
        if history[h][2] == 0 and histroy[h][1] == 0:
            mdl.add(w[j] != historylist[h][i] for i in range(5) for j in range(5)) # Have new guess with all different letters than the old one
    for h in history:
        if h[1]!= 0:
            mdl.add(mdl.sum((w[i] + 97) == ord(h[0][i]) for i in range(5)) == h[1]) # For nbCorrect
    for h in history:
        if h[2] != 0:
            wordd = [ord(i-97) for i in h[0]]
            mdl.add(mdl.sum(mdl.count(w,ord(h[0][i]) - 97)/mdl.count(wordd,ord(h[0][i]) - 97) for i in range(5)) >= h[1] + h[2]) # For nbPresent
            
    slist = []
    sol = mdl.solve()
    Solutions = sol.get_all_var_solutions() # Retrieve all the solutions
    
    for s in Solutions:
        slist.append(s.get_value())
    
    guess = convert_array_to_string([chr(i + 97) for i in slist]) # Return guess value
    return guess
    

In [239]:
# this function returns a secret word from the set of Words
def select_secret_word(Words):
    return random.sample(Words,1)[0]

In [240]:
# this function computes the score (nb_correct,nb_present) for the guess given the secret word

def compute_scores(secret,guess):
    nbCorrect = 0
    nbPresent = 0
    for i in range(5):
        if guess[i] == secret[i]:
            nbCorrect = nbCorrect + 1
        if guess.count(guess[i]) == 1 and guess[i] in secret and guess[i] != secret[i]:
            nbPresent = nbPresent + 1
    return (nbCorrect,nbPresent)

In [241]:
# this function plays the game with the list of words in the file name
def play_wordle(fileName):
    Words = get_set_of_words(fileName)
    secret = select_secret_word(Words)
    print("secret:",secret)
    history = []
    i = 0
    while True:
        g = guess(Words,history)
        print("guess :",g) 
        (nbCorrect,nbPresent) = compute_scores(secret,g)
        print("\t correct:",nbCorrect," present:",nbPresent)
        if nbCorrect == len(secret):
            break
        history.append((g,nbCorrect,nbPresent))

In [242]:
play_wordle('5-letter-words.csv')

secret: laugh
 ! --------------------------------------------------- CP Optimizer 22.1.0.0 --
 ! Satisfiability problem - 5 variables, 2 constraints
 ! Initial process time : 0.02s (0.02s extraction + 0.00s propagation)
 !  . Log search space  : 23.2 (before), 23.2 (after)
 !  . Memory usage      : 1.3 MB (before), 1.3 MB (after)
 ! Using parallel search with 12 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
 *                      4  0.05s        1        18  = w_0
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 98
 ! Number of fails        : 30
 ! Total memory usage     : 19.6 MB (19.6 MB CP Optimizer + 0.0 MB Concert)
 ! Time spent in solve    : 0.05s (0.04s engine + 0.02s extraction)
 ! Search speed (b

CpoException: Impossible to build a tuple set from value '[18, 20, 13, 13, 24, []]' of type '<class 'list'>'